In [32]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [34]:

#------------------------------------------------------------------------------
# THIS DATA IS NOT CONTINUED FROM THE PREV LAB FOR SIMPLICITY
#------------------------------------------------------------------------------

df = pd.read_csv('bangkok_traffy.csv')
print(df.shape)
df.head()

(787026, 16)


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00


In [37]:
traffy = df.drop(['ticket_id', 'organization', 'photo', 'photo_after',
            'address', 'state', 'star', 'count_reopen', 'last_activity'], axis=1)

In [39]:
null_counts = traffy.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
type             97
comment        8772
coords            0
subdistrict     566
district        561
province        195
timestamp         0
dtype: int64


In [ ]:
traffy['coords'].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787026 entries, 0 to 787025
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   type         786929 non-null  object
 1   comment      778254 non-null  object
 2   coords       787026 non-null  object
 3   subdistrict  786460 non-null  object
 4   district     786465 non-null  object
 5   province     786831 non-null  object
 6   timestamp    787026 non-null  object
dtypes: object(7)
memory usage: 42.0+ MB


In [ ]:
# 1. Select numeric columns only
numeric_cols = traffy.select_dtypes(include=['number']).columns.tolist()
categorical_cols = [col for col in traffy.columns if col not in numeric_cols]

# 2. Define mean imputation transformer for numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])
# 3. Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols)
    ],
    remainder='passthrough'  # keep non-numeric columns unchanged
)

# 4. Apply the pipeline
traffy_imputed = preprocessor.fit_transform(traffy)

# 5. Reconstruct the DataFrame
traffy_imputed = pd.DataFrame(traffy_imputed, columns=numeric_cols + categorical_cols)
traffy_imputed = traffy_imputed[traffy.columns.tolist()]

In [ ]:
null_counts = traffy_imputed.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
ticket_id          8772
type                 97
organization        571
comment            8772
photo               115
photo_after      145717
coords                0
address            8772
subdistrict         566
district            561
province            195
timestamp             0
state                 0
star                  0
count_reopen          0
last_activity         0
dtype: int64


In [ ]:
# 1. Identify column types
numeric_cols = traffy.select_dtypes(include=['number']).columns.tolist()
categorical_cols = [col for col in traffy.columns if col not in numeric_cols]

# 2. Define transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# 3. ColumnTransformer for both
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# 4. Fit and transform
traffy_imputed = preprocessor.fit_transform(traffy)

# 5. Reconstruct DataFrame with original column order
traffy_imputed = pd.DataFrame(traffy_imputed, columns=numeric_cols + categorical_cols)
traffy_imputed = traffy_imputed[traffy.columns.tolist()]

In [ ]:
null_counts = traffy_imputed.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
ticket_id        0
type             0
organization     0
comment          0
photo            0
photo_after      0
coords           0
address          0
subdistrict      0
district         0
province         0
timestamp        0
state            0
star             0
count_reopen     0
last_activity    0
dtype: int64
